In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta

# On lit et on met en forme le dataframe
# Séparation des colonnes csv, renommage des colonnes, suppression de la première ligne comme header
df = pd.read_csv("data/eurusd_4h.csv",  sep="\t", names=['hour', 'open', 'high', 'low', 'close', 'tick', 'volume', 'spread'], header=0)

# On type la colonne date en datetime
df['datetime'] = pd.to_datetime(df.index + ' ' + df['hour'])
df['datetime'] = pd.to_datetime(df['datetime'],  format='%Y.%m.%d %H:%M')

# On set l'index à la colonne date
df.set_index('datetime', inplace=True)

# On garde uniquement la colonne close pour la stratégie
df = df[[ 'close']]

# Calcul du RSI
df['rsi'] = ta.rsi(df['close'], length=14)

df['signal'] = 0

# Signal d'achat
cross_up = (df['rsi'].shift(1) < 30) & (df['rsi'] > 30)
df.loc[cross_up, 'signal'] = 1
# Signal de vente
cross_down = (df['rsi'].shift(1) > 70) & (df['rsi'] < 70)
df.loc[cross_down, 'signal'] = -1

# On récupère le % de variation entre chaque trade
df['returns'] = df['close'].pct_change()

# On supprimes les lignes qui posssèdent des valeurs nulles
df = df.dropna()

df['position'] = 0

# On prend position à l'achat
df.loc[df['signal'] == 1, 'position'] = 1
# On ferme la position à l'achat
df.loc[df['signal'] == -1, 'position'] = 0

# On remplie les valeurs vide par la précédente et les NaN par des 0 (pas de position)
df['position'] = df['position'].ffill().fillna(0)

# Calcul du rendement par position
df['strategy_returns'] = df['position'] * df['returns']

# plt.figure(figsize=(14, 6))
# plt.plot(df.index, df['close'], label='Cours de clôture', color='black')

# Visualisation du RSI
# plt.plot(df.index, df['rsi'], label='RSI 14', color='purple')
# plt.axhline(70, linestyle='--', color='red', label='Surachat (70)')
# plt.axhline(30, linestyle='--', color='green', label='Survente (30)')

# Signaux d'achat (RSI croise 30 vers le haut)
# plt.scatter(df.index[df['position'] == 1], df['close'][df['position'] == 1],
#             marker='^', color='green', label='Signal Achat')

# # Signaux de vente (RSI croise 70 vers le bas)
# exit_signals = (df['position'].shift(1) == 1) & (df['position'] == 0)
# plt.scatter(df.index[exit_signals], df['close'][exit_signals],
#             marker='x', color='red', label='Signal Vente')

# plt.legend()
# plt.grid(True)
# plt.show()

sum_trades = df[df['position'] == 1]['position'].sum()
sum_win_trades = df[(df['position'] == 1) & (df['returns'] > 0)]['position'].sum()
winrate = (1 + df['strategy_returns']).prod() - 1

print(f"Trades totaux : {sum_trades}")
print(f"Trades gagnants : {sum_win_trades}")
print(f"Gain total : {winrate:.2f}")
print(f"Pourcentage de gain : {((sum_trades / sum_win_trades) * 100):.2f}%")



Trades totaux : 32
Trades gagnants : 32
Gain total : 0.04
Pourcentage de gain : 100.00%
